In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models, tools
from biogeme.expressions import Beta, Variable, bioDraws, MonteCarlo, log, Power, exp, Derive, RandomVariable
import biogeme.distributions as dist
import scipy.stats as st

Data description

The dataset `apollo_swissRouteChoiceData` comes from an actual SP survey of public transport route choice conducted in
Switzerland (Axhausen et al., 2008). A set of 388 people were faced with 9 choices each between two public transport routes, both using train (leading to 3'492 observations in the data). The two alternatives are described on the basis of travel time, travel cost, headway (time between subsequent trains/busses) and the number of interchanges. For each individual, the dataset
additionally contains information on income, car availability in the household, and whether the
journey was made for commuting, shopping, business or leisure.

See slide 79 Apollo Documentation for more info on Mixtures

Preparing the data

In [ ]:
# Read the CSV file
df = pd.read_csv(r"C:\Users\alexi\Desktop\GITS\Semester-project\Data\apollo_swissRouteChoiceData.csv")
df

In [ ]:
#fill nAn values with 0 (same as in Apollo)
df = df.fillna(0) 

#creating biogeme database

database = db.Database('MMNL_Preference_Model', df)
globals().update(database.variables) #transform all columns into variables

Defining the model parameters

In [ ]:
mu_log_b_tt = Beta('mu_log_b_tt', -3, None, None, 0)
sigma_log_b_tt = Beta('sigma_log_b_tt', -0.01, None, None, 0)
mu_log_b_tc = Beta('mu_log_b_tc', -3, None, None, 0)
sigma_log_b_tc = Beta('sigma_log_b_tc', -0.01, None, None, 0)
mu_log_b_hw = Beta('mu_log_b_hw', -3, None, None, 0)
sigma_log_b_hw = Beta('sigma_log_b_hw', -0.01, None, None, 0)
mu_log_b_tt = Beta('mu_log_b_tt', -3, None, None, 0)
sigma_log_b_ch = Beta('sigma_log_b_ch', -0.01, None, None, 0)

#no fixed parameters



In [ ]:
help(bioDraws)

Generate draws from Random Distribution (Halton Draws)

In [ ]:
b_tt = -np.exp(mu_log_b_tt + sigma_log_b_tt*bioDraws('b_tt', 'NORMAL_HALTON5'))
b_tc = -np.exp(mu_log_b_tc + sigma_log_b_tc*bioDraws('b_tc', 'NORMAL_HALTON5'))
b_hw = -np.exp(mu_log_b_hw + sigma_log_b_hw*bioDraws('b_hw', 'NORMAL_HALTON5'))
b_ch = -np.exp(mu_log_b_ch + sigma_log_b_ch*bioDraws('b_ch', 'NORMAL_HALTON5'))



Defining the Model

In [ ]:
V_1 = b_tt * tt1 + b_tc * tc1 + b_hw * hw1 + b_ch * ch1
V_2 = b_tt * tt2 + b_tc * tc2 + b_hw * hw2 + b_ch * ch2

av = {1:av_1, 2:av_2}

V = {1: V_1, 2: V_2}

Estimating the model

In [ ]:
# Define the model
logprob_MMNL = log(MonteCarlo(models.loglogit(V, av, choice)))

#call cpu/ram function

# Estimate the model
biogeme_MMNL = bio.BIOGEME(database, logprob_MMNL)
biogeme_MMNL.modelName = 'MMNL_Model'
biogeme_MMNL.generateHtml = True  # Disable HTML file generation
biogeme_MMNL.generatePickle = False  # Disable PICKLE file generation
biogeme_MMNL.save_iterations = False  # Disable ITER file generation
results_model_MMNL = biogeme_MMNL.estimate()

# Output
print(results_model_MMNL.getEstimatedParameters())

In [ ]:
# Retrieve the general statistics from the results
general_stats_model_RP = results_model_RP.getGeneralStatistics()
print(results_model_RP.printGeneralStatistics())